# 多层感知机（MLP）

*  使用 tf.keras.datasets 获得数据集并预处理
*  使用 tf.keras.Model 和 tf.keras.layers 构建模型
*  构建模型训练流程，使用 tf.keras.losses 计算损失函数，并使用 tf.keras.optimizer 优化模型
*  构建模型评估流程，使用 tf.keras.metrics 计算评估指标





In [13]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


In [3]:
!pip uninstall tensorflow

Uninstalling tensorflow-1.15.0rc3:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0rc3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0rc3


In [4]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 386kB/s 
     |████████████████████████████████| 450kB 34.9MB/s 
     |████████████████████████████████| 3.8MB 29.5MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [14]:
print(tf.__version__)

2.0.0


In [15]:
tf.executing_eagerly() 
#tf.enable_eager_execution()

True

# 数据获取及预处理： tf.keras.datasets
先进行预备工作，实现一个简单的 MNISTLoader 类来读取 MNIST 数据集数据。这里使用了 tf.keras.datasets 快速载入 MNIST 数据集。

In [0]:
class MNISTLoader():
  def __init__(self):
    mnist = tf.keras.datasets.mnist
    (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
    self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)
    self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)
    self.train_label = self.train_label.astype(np.int32)    # [60000]
    self.test_label = self.test_label.astype(np.int32)      # [10000]
    self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
  
  def get_batch(self, batch_size):
    index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
    return self.train_data[index, :], self.train_label[index]
    

# 模型的构建： tf.keras.Model 和 tf.keras.layers
多层感知机的模型类实现与上面的线性模型类似，使用 tf.keras.Model 和 tf.keras.layers 构建，所不同的地方在于层数增加了（顾名思义，“多层” 感知机），以及引入了非线性激活函数（这里使用了 ReLU 函数 ， 即下方的 activation=tf.nn.relu ）。该模型输入一个向量（比如这里是拉直的 1×784 手写体数字图片），输出 10 维的向量，分别代表这张图片属于 0 到 9 的概率。



In [0]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(units= 100, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units = 10)
    
  def call(self, inputs):
    x = self.flatten(inputs)
    x = self.dense1(x)
    x = self.dense2(x)
    output = tf.nn.softmax(x)
    return output

# 模型的训练： tf.keras.losses 和 tf.keras.optimizer


In [0]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [0]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

然后迭代进行以下步骤：

从 DataLoader 中随机取一批训练数据；

将这批数据送入模型，计算出模型的预测值；

将模型预测值与真实值进行比较，计算损失函数（loss）。这里使用 tf.keras.losses 中的交叉熵函数作为损失函数；

计算损失函数关于模型变量的导数；

将求出的导数值传入优化器，使用优化器的 apply_gradients 方法更新模型参数以最小化损失函数

In [20]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
  X, y = data_loader.get_batch(batch_size)
  with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
    loss = tf.reduce_mean(loss)
    print("batch %d: loss %f" % (batch_index, loss.numpy()))
  grads = tape.gradient(loss, model.variables)
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.285256
batch 1: loss 2.205372
batch 2: loss 2.334929
batch 3: loss 2.056121
batch 4: loss 2.081305
batch 5: loss 1.941437
batch 6: loss 2.061041
batch 7: loss 1.972376
batch 8: loss 1.754983
batch 9: loss 1.749182
batch 10: loss 1.632017
batch 11: loss 1.592685
batch 12: loss 1.543856
batch 13: loss 1.629244
batch 14: loss 1.514880
batch 15: loss 1.428778
batch 16: loss 1.384761
batch 17: loss 1.380737
batch 18: loss 1.174533
batch 19: loss 1.367718
batch 20: loss 1.302076
batch 21: loss 1.189340
batch 22: loss 1.042583
batch 23: loss 1.025285
batch 24: loss 1.220943
batch 25: loss 1.032526
batch 26: loss 0.888587
batch 27: loss 1.059231
batch 28: loss 1.008494
batch 29: loss 0.836554
batch 30: loss 0.864243
batch 31: loss 0.854915
batch 32: loss 0.907002
batch 33: loss 1.022292
batch 34: loss 0.848349
batch 35: loss 0.761802
batch 36: loss 0.874089
batch 37: loss 0.739989
batch 38: loss 0.718120
batch 39: loss 0.734513
batch 40: loss 0.776238
batch 41: loss 0.595228
ba

# 模型的评估： tf.keras.metrics
最后，我们使用测试集评估模型的性能。这里，我们使用 tf.keras.metrics 中的 SparseCategoricalAccuracy 评估器来评估模型在测试集上的性能，该评估器能够对模型预测的结果与真实结果进行比较，并输出预测正确的样本数占总样本数的比例。我们迭代测试数据集，每次通过 update_state() 方法向评估器输入两个参数： y_pred 和 y_true ，即模型预测出的结果和真实结果。评估器具有内部变量来保存当前评估指标相关的参数数值（例如当前已传入的累计样本数和当前预测正确的样本数）。迭代结束后，我们使用 result() 方法输出最终的评估指标值（预测正确的样本数占总样本数的比例）。

在以下代码中，我们实例化了一个 tf.keras.metrics.SparseCategoricalAccuracy 评估器，并使用 For 循环迭代分批次传入了测试集数据的预测结果与真实结果，并输出训练后的模型在测试数据集上的准确率。



In [21]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
  start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
  y_pred = model.predict(data_loader.test_data[start_index: end_index])
  sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index:end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.973400
